In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../')

In [3]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *

In [4]:
# Location of your midi filesfiles
midi_path = Path('data/midi/examples')
data_path = Path('data/numpy')
data_save_name = 'musicitem_data_save.pkl'

In [5]:
midi_files = get_files(midi_path, '.mid', recurse=True)
data = MusicDataBunch.from_files(midi_files, data_path, processors=[Midi2ItemProcessor()], bs=4, bptt=128,
                                 encode_position=True, dl_tfms=[batch_position_tfm])

In [6]:
config = default_config()
config['encode_position'] = True
config['transpose_range'] = (0, 12)
config['mask_steps'] = 4

In [7]:
model = get_language_model(arch=MusicTransformerXL, vocab_sz=len(data.vocab), config=config.copy())

In [8]:
learn = MusicLearner(data, model)

In [9]:
learn.to_fp16(dynamic=True, clip=0.5);

In [10]:
learn.fit_one_cycle(4)

epoch,train_loss,valid_loss,accuracy,time
0,3.891204,2.633192,0.375000,00:02
1,3.757607,2.668247,0.386719,00:01
2,3.506466,2.503491,0.125000,00:01
3,3.403756,2.606918,0.125000,00:01


## 5. Predict

In [ ]:
midi_file = Path('data/midi/notebook_examples/single_bar_example.mid')
item = MusicItem.from_file(midi_file, data.vocab);
pred, full = learn.predict(item, n_words=100)
pred.show()

In [ ]:
pred.play()